In [1]:
import reframed
import sys
sys.path.append('../code/')
from pathlib import Path
import pandas as pd
import utils
from dFBA_reframed import dFBA
import numpy as np

In [2]:
growth_rates_fn = '/Users/ssulheim/Library/CloudStorage/OneDrive-SINTEF/UNIL/MWF/experimental work/230525_growth_phenotyping/fitted_growth_parameters.csv'

In [3]:
growth_rates_df = pd.read_csv(growth_rates_fn, index_col=0)

In [4]:
def set_auxotrophy_dict(auxotrophy_dict):
    constraints_dict = {}
    for species_abbr, species_dict in auxotrophy_dict.items():
        species_constraint_dict = {}
        if species_dict.get('vitamins'):
            for met_id in species_dict['vitamins']:
                r_id = f'R_EX_{met_id}_e'
                species_constraint_dict[r_id] = (-0.001, 0)
        if species_dict.get('amino acids'):
            for met_id in species_dict['amino acids']:
                r_id = f'R_EX_{met_id}_e'
                species_constraint_dict[r_id] = (-0.2, 0)
        constraints_dict[species_abbr] = species_constraint_dict
    return constraints_dict

In [5]:
cs = 'Acetate'
growth_rate_dict = {}
for species_abbr in growth_rates_df.Species.unique():
    idx = (growth_rates_df['Species']==species_abbr) & (growth_rates_df['Carbon source']==cs)
    growth_rate_dict[species_abbr] = growth_rates_df.loc[idx, 'r'].values[0]

In [6]:
model_folder = Path('/Users/ssulheim/git/mwf_gems/models/gf_models/carveme_TFA')
model1_fn = model_folder / 'polished_At.xml'
model2_fn = model_folder / 'polished_Ct.xml'
model3_fn = model_folder / 'polished_Oa.xml'
model4_fn = model_folder / 'polished_Ml.xml'

model_name_dict = {}
yield_dict = {}
vmax_dict = {}
for fn, name in zip([model1_fn, model2_fn, model3_fn, model4_fn],['At', 'Ct', 'Oa', 'Ml']):
    model = reframed.load_cbmodel(fn)
    model.reactions.R_EX_h_e.lb = -1000
    model.reactions.R_EX_o2_e.lb = -1000
    model_name_dict[name] = [model, 0.01]
    model.solver = 'gurobi'
    fba_result = reframed.FBA(model, constraints={'R_EX_glc__D_e':0, 'R_EX_cit_e':0, 'R_EX_ac_e':-10})
    yield_dict[name] = fba_result.fobj/10
    vmax_dict[name] = growth_rate_dict[name]/yield_dict[name]


Set parameter Username


2023-11-19 23:56:47 INFO:gurobipy.gurobipy Set parameter Username


Academic license - for non-commercial use only - expires 2024-02-26


2023-11-19 23:56:47 INFO:gurobipy.gurobipy Academic license - for non-commercial use only - expires 2024-02-26


In [7]:
auxotrophy_dict = {
    'Ml': {'amino acids': ['pro__L', 'cys__L'], 'vitamins':['thm', 'btn']},
    'Oa': {'vitamins': ['thm']}
    }
auxotrophy_constraints = set_auxotrophy_dict(auxotrophy_dict)

In [9]:
N_runs = 1
for j in range(N_runs):
    cs_formula = model.metabolites['M_ac_c'].metadata['FORMULA']
    ac_mM = utils.convert_gL_to_mM(cs_formula, 2)
    dt = 0.4
    total_time = 58
    iterations = int(total_time//dt)
    dilution_interval = 20
    D = dFBA(iterations = iterations, dt = dt, method = "FBA_with_leakage", store_exchanges_flag = False, fraction_of_optimum=0.9, 
             dilution_interval=dilution_interval, folder = 'dFBA_191123')
    D.medium.define_initial_conditions({"M_ac_e": ac_mM})
    D.add_models(model_name_dict, auxotrophy_constraints=auxotrophy_constraints)

    # Set Km and vMax
    noise = np.random.normal(0, 1, len(model_name_dict))
    for i, name in enumerate(model_name_dict.keys()):
        D.models[name].set_km("M_ac_e", 1)
        D.models[name].set_Vmax("M_ac_e", max(0, vmax_dict[name]+noise[i]))

    # D.medium.set_store_concentrations(["glc__D_e", "nh3_e"])
    D.run()
    # D.store_results(True)
    D.store_results(True, tag = j+1)

2023-11-19 23:58:43 INFO:Model At Use FBA with leakage


Candidate metabolites 53:
['M_2pg_c', 'M_3pg_c', 'M_4ahmmp_c', 'M_4hba_c', 'M_ac_c', 'M_acald_c', 'M_adn_c', 'M_akg_c', 'M_ala__L_c', 'M_arg__L_c', 'M_asn__L_c', 'M_asp__L_c', 'M_cit_c', 'M_cys__L_c', 'M_dhap_c', 'M_e4p_c', 'M_f6p_c', 'M_fdp_c', 'M_fe3dcit_c', 'M_for_c', 'M_fum_c', 'M_g3p_c', 'M_g6p_c', 'M_gln__L_c', 'M_glu__L_c', 'M_gly_c', 'M_his__L_c', 'M_hom__L_c', 'M_icit_c', 'M_ile__L_c', 'M_inost_c', 'M_leu__L_c', 'M_lys__L_c', 'M_mal__L_c', 'M_met__L_c', 'M_oaa_c', 'M_orn_c', 'M_orot_c', 'M_pep_c', 'M_phe__L_c', 'M_ppa_c', 'M_pro__L_c', 'M_pyr_c', 'M_r5p_c', 'M_ru5p__D_c', 'M_ser__L_c', 'M_skm_c', 'M_succ_c', 'M_thr__L_c', 'M_trp__L_c', 'M_tyr__L_c', 'M_val__L_c', 'M_xu5p__D_c']


2023-11-19 23:58:44 INFO:Model Ct Use FBA with leakage


At Leak exchanges: 
['R_EX_2pg_e', 'R_EX_3pg_e', 'R_EX_4hba_e', 'R_EX_acald_e', 'R_EX_akg_e', 'R_EX_ala__L_e', 'R_EX_arg__L_e', 'R_EX_asn__L_e', 'R_EX_asp__L_e', 'R_EX_cit_e', 'R_EX_cys__L_e', 'R_EX_dhap_e', 'R_EX_e4p_e', 'R_EX_f6p_e', 'R_EX_fdp_e', 'R_EX_fe3dcit_e', 'R_EX_fum_e', 'R_EX_g3p_e', 'R_EX_g6p_e', 'R_EX_gln__L_e', 'R_EX_glu__L_e', 'R_EX_gly_e', 'R_EX_his__L_e', 'R_EX_hom__L_e', 'R_EX_icit_e', 'R_EX_ile__L_e', 'R_EX_leu__L_e', 'R_EX_lys__L_e', 'R_EX_met__L_e', 'R_EX_oaa_e', 'R_EX_orot_e', 'R_EX_pep_e', 'R_EX_phe__L_e', 'R_EX_pro__L_e', 'R_EX_pyr_e', 'R_EX_r5p_e', 'R_EX_ru5p__D_e', 'R_EX_ser__L_e', 'R_EX_skm_e', 'R_EX_succ_e', 'R_EX_thr__L_e', 'R_EX_trp__L_e', 'R_EX_tyr__L_e', 'R_EX_val__L_e', 'R_EX_xu5p__D_e']
Candidate metabolites 53:
['M_2pg_c', 'M_34dhbz_c', 'M_3pg_c', 'M_4hba_c', 'M_4hbz_c', 'M_ac_c', 'M_adn_c', 'M_akg_c', 'M_ala_B_c', 'M_ala__L_c', 'M_alaala_c', 'M_arg__L_c', 'M_asn__L_c', 'M_asp__L_c', 'M_cit_c', 'M_cys__L_c', 'M_dhap_c', 'M_e4p_c', 'M_f6p_c', 'M_fdp_c'

2023-11-19 23:58:45 INFO:Model Oa Use FBA with leakage


Ct Leak exchanges: 
['R_EX_2pg_e', 'R_EX_3pg_e', 'R_EX_4hba_e', 'R_EX_akg_e', 'R_EX_ala__L_e', 'R_EX_alaala_e', 'R_EX_arg__L_e', 'R_EX_asn__L_e', 'R_EX_asp__L_e', 'R_EX_cit_e', 'R_EX_cys__L_e', 'R_EX_dhap_e', 'R_EX_e4p_e', 'R_EX_f6p_e', 'R_EX_fdp_e', 'R_EX_fum_e', 'R_EX_g3p_e', 'R_EX_gln__L_e', 'R_EX_glu__L_e', 'R_EX_gly_e', 'R_EX_his__L_e', 'R_EX_hom__L_e', 'R_EX_icit_e', 'R_EX_ile__L_e', 'R_EX_leu__L_e', 'R_EX_lys__L_e', 'R_EX_met__L_e', 'R_EX_oaa_e', 'R_EX_orot_e', 'R_EX_pep_e', 'R_EX_phe__L_e', 'R_EX_pro__L_e', 'R_EX_pyr_e', 'R_EX_r5p_e', 'R_EX_ser__L_e', 'R_EX_skm_e', 'R_EX_succ_e', 'R_EX_thr__L_e', 'R_EX_trp__L_e', 'R_EX_tyr__L_e', 'R_EX_val__L_e', 'R_EX_xu5p__D_e']
Candidate metabolites 54:
['M_2pg_c', 'M_3pg_c', 'M_4ahmmp_c', 'M_ac_c', 'M_adn_c', 'M_akg_c', 'M_ala_B_c', 'M_ala__L_c', 'M_arg__L_c', 'M_asn__L_c', 'M_asp__L_c', 'M_cit_c', 'M_cys__L_c', 'M_dhap_c', 'M_e4p_c', 'M_f6p_c', 'M_fdp_c', 'M_fe3dcit_c', 'M_for_c', 'M_fum_c', 'M_g3p_c', 'M_g6p_c', 'M_gln__L_c', 'M_glu__L_c'

2023-11-19 23:58:46 INFO:Model Ml Use FBA with leakage


Oa Leak exchanges: 
['R_EX_2pg_e', 'R_EX_3pg_e', 'R_EX_akg_e', 'R_EX_ala__L_e', 'R_EX_arg__L_e', 'R_EX_asn__L_e', 'R_EX_asp__L_e', 'R_EX_cit_e', 'R_EX_cys__L_e', 'R_EX_dhap_e', 'R_EX_e4p_e', 'R_EX_f6p_e', 'R_EX_fdp_e', 'R_EX_fum_e', 'R_EX_g3p_e', 'R_EX_g6p_e', 'R_EX_gln__L_e', 'R_EX_glu__L_e', 'R_EX_gly_e', 'R_EX_his__L_e', 'R_EX_hom__L_e', 'R_EX_icit_e', 'R_EX_ile__L_e', 'R_EX_leu__L_e', 'R_EX_lys__L_e', 'R_EX_met__L_e', 'R_EX_oaa_e', 'R_EX_orot_e', 'R_EX_pep_e', 'R_EX_phe__L_e', 'R_EX_pro__L_e', 'R_EX_pyr_e', 'R_EX_r5p_e', 'R_EX_ru5p__D_e', 'R_EX_ser__L_e', 'R_EX_skm_e', 'R_EX_succ_e', 'R_EX_thr__L_e', 'R_EX_trp__L_e', 'R_EX_tyr__L_e', 'R_EX_val__L_e', 'R_EX_xu5p__D_e']
Candidate metabolites 52:
['M_2pg_c', 'M_3pg_c', 'M_4ahmmp_c', 'M_ac_c', 'M_adn_c', 'M_akg_c', 'M_ala_B_c', 'M_ala__D_c', 'M_ala__L_c', 'M_arg__L_c', 'M_asn__L_c', 'M_asp__L_c', 'M_cit_c', 'M_cys__L_c', 'M_dhap_c', 'M_e4p_c', 'M_f6p_c', 'M_for_c', 'M_fum_c', 'M_g3p_c', 'M_gln__L_c', 'M_glu__L_c', 'M_gly_c', 'M_glyc3p_

2023-11-19 23:58:48 INFO:Medium   Model biomass At: 0.010353075060025706, growth rate: 0.08674622459624848
2023-11-19 23:58:49 INFO:Medium   Model biomass Ct: 0.01053282205142151, growth rate: 0.12977799582415334
2023-11-19 23:58:49 INFO:Medium   Model biomass Oa: 0.01063496238074416, growth rate: 0.15390454585615446
2023-11-19 23:58:49 INFO:Medium   Model biomass Ml: 0.01001336931376123, growth rate: 0.003340096197775147
2023-11-19 23:58:50 INFO:Medium   Model biomass Oa: 0.011311584718846373, growth rate: 0.1542012139481098
2023-11-19 23:58:51 INFO:Medium   Model biomass At: 0.010726576854136447, growth rate: 0.08860224300203724
2023-11-19 23:58:51 INFO:Medium   Model biomass Ct: 0.011099107273263588, growth rate: 0.1309209698153448
2023-11-19 23:58:51 INFO:Medium   Model biomass Ml: 0.01003931746810328, growth rate: 0.006469998037457297
2023-11-19 23:58:52 INFO:Medium   Model biomass Oa: 0.01203463742231026, growth rate: 0.15490386729854666
2023-11-19 23:58:53 INFO:Medium   Model bi

dFBA simulation took 306 seconds


In [62]:
iterations

117.5

In [7]:
Oa = model_name_dict['Oa'][0]

In [10]:
Oa.reactions.Growth.stoichiometry['M_thmpp_c']

-0.000216